# $$Empirical \text{ } Finance$$
## Portfolio Returns Assignment: 
### _Stock Split Annoucements_

Rohitashwa Chakraborty _(rc47878)_

In [7]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import statsmodels.api as sm
from patsy import dmatrices
import yfinance as yf
from datetime import datetime

import wrds
connection = wrds.Connection()

from tqdm.notebook import tqdm
tqdm.pandas()

%matplotlib inline
plt.style.use("bmh")

Loading library list...
Done


# Table Description

## Compustat
comp/secd

<table id="data-table" class="table table-bordered table-hover table-striped table-mobile tablesorter tablesorter-default">
<thead>
<tr>
<th class="text-nowrap">Variable Name</th>
<th>Type</th>
<th>Length</th>
<th>Description</th>
</tr>
</thead>
<tbody>
<tr>
<td style="background-color: inherit;">adrrc</td>
<td style="background-color: inherit;">double</td>
<td style="background-color: inherit;">53</td>
<td style="background-color: inherit;">ADR Ratio - Daily</td>
</tr>
<tr>
<td style="background-color: inherit;">ajexdi</td>
<td style="background-color: inherit;">double</td>
<td style="background-color: inherit;">53</td>
<td style="background-color: inherit;">Adjustment Factor (Issue)-Cumulative by Ex-Date</td>
</tr>
<tr>
<td style="background-color: inherit;">
anncdate
<a href="javascript:showMe(&quot;/data-dictionary/comp_na_daily_all/secd/anncdate/&quot;);">
<span class="fa fa-question-circle float-right"></span>
</a>
</td>
<td style="background-color: inherit;">date</td>
<td style="background-color: inherit;"></td>
<td style="background-color: inherit;">
Dividend Declaration Date
</td>
</tr>
<tr>
<td style="background-color: inherit;">
capgn
<a href="javascript:showMe(&quot;/data-dictionary/comp_na_daily_all/secd/capgn/&quot;);">
<span class="fa fa-question-circle float-right"></span>
</a>
</td>
<td style="background-color: inherit;">double</td>
<td style="background-color: inherit;">53</td>
<td style="background-color: inherit;">
Capital Gains - Daily
</td>
</tr>
<tr>
<td style="background-color: inherit;">
capgnpaydate
<a href="javascript:showMe(&quot;/data-dictionary/comp_na_daily_all/secd/capgnpaydate/&quot;);">
<span class="fa fa-question-circle float-right"></span>
</a>
</td>
<td style="background-color: inherit;">date</td>
<td style="background-color: inherit;"></td>
<td style="background-color: inherit;">
Capital Gains Payment Date
</td>
</tr>

<tr>
<td style="background-color: inherit;">
cheqv


<a href="javascript:showMe(&quot;/data-dictionary/comp_na_daily_all/secd/cheqv/&quot;);"><span class="fa fa-question-circle float-right"></span></a>


</td>
<td style="background-color: inherit;">double</td>
<td style="background-color: inherit;">53</td>
<td style="background-color: inherit;">
Cash Equivalent Distributions
</td>
</tr>

<tr>
<td style="background-color: inherit;">
cheqvpaydate
<a href="javascript:showMe(&quot;/data-dictionary/comp_na_daily_all/secd/cheqvpaydate/&quot;);"><span class="fa fa-question-circle float-right"></span></a>
</td>
<td style="background-color: inherit;">date</td>
<td style="background-color: inherit;"></td>
<td style="background-color: inherit;">
Cash Equivalent Distributions per Share Payment Date
</td>
</tr>
<tr>
<td style="background-color: inherit;">
cik
<a href="javascript:showMe(&quot;/data-dictionary/comp_na_daily_all/secd/cik/&quot;);"><span class="fa fa-question-circle float-right"></span></a>
</td>
<td style="background-color: inherit;">string</td>
<td style="background-color: inherit;">10</td>
<td style="background-color: inherit;">
CIK Number
</td>
</tr>

<tr>
<td style="background-color: inherit;">
conm


<a href="javascript:showMe(&quot;/data-dictionary/comp_na_daily_all/secd/conm/&quot;);"><span class="fa fa-question-circle float-right"></span></a>


</td>
<td style="background-color: inherit;">string</td>
<td style="background-color: inherit;">57</td>
<td style="background-color: inherit;">
Company Name
</td>
</tr>

<tr>
<td style="background-color: inherit;">
cshoc


<a href="javascript:showMe(&quot;/data-dictionary/comp_na_daily_all/secd/cshoc/&quot;);"><span class="fa fa-question-circle float-right"></span></a>


</td>
<td style="background-color: inherit;">double</td>
<td style="background-color: inherit;">53</td>
<td style="background-color: inherit;">
Shares Outstanding
</td>
</tr>

<tr>
<td style="background-color: inherit;">
cshtrd


<a href="javascript:showMe(&quot;/data-dictionary/comp_na_daily_all/secd/cshtrd/&quot;);"><span class="fa fa-question-circle float-right"></span></a>


</td>
<td style="background-color: inherit;">double</td>
<td style="background-color: inherit;">53</td>
<td style="background-color: inherit;">
Trading Volume - Daily
</td>
</tr>

<tr>
<td style="background-color: inherit;">
curcdd


<a href="javascript:showMe(&quot;/data-dictionary/comp_na_daily_all/secd/curcdd/&quot;);"><span class="fa fa-question-circle float-right"></span></a>


</td>
<td style="background-color: inherit;">string</td>
<td style="background-color: inherit;">3</td>
<td style="background-color: inherit;">
ISO Currency Code - Daily
</td>
</tr>

<tr>
<td style="background-color: inherit;">
curcddv


<a href="javascript:showMe(&quot;/data-dictionary/comp_na_daily_all/secd/curcddv/&quot;);"><span class="fa fa-question-circle float-right"></span></a>


</td>
<td style="background-color: inherit;">string</td>
<td style="background-color: inherit;">3</td>
<td style="background-color: inherit;">
ISO Currency Code - Dividend
</td>
</tr>

<tr>
<td style="background-color: inherit;">
cusip


<a href="javascript:showMe(&quot;/data-dictionary/comp_na_daily_all/secd/cusip/&quot;);"><span class="fa fa-question-circle float-right"></span></a>


</td>
<td style="background-color: inherit;">string</td>
<td style="background-color: inherit;">9</td>
<td style="background-color: inherit;">
CUSIP
</td>
</tr>

<tr>
<td style="background-color: inherit;">
datadate


<a href="javascript:showMe(&quot;/data-dictionary/comp_na_daily_all/secd/datadate/&quot;);"><span class="fa fa-question-circle float-right"></span></a>


</td>
<td style="background-color: inherit;">date</td>
<td style="background-color: inherit;"></td>
<td style="background-color: inherit;">
Data Date - Dividends
</td>
</tr>

<tr>
<td style="background-color: inherit;">
div


<a href="javascript:showMe(&quot;/data-dictionary/comp_na_daily_all/secd/div/&quot;);"><span class="fa fa-question-circle float-right"></span></a>


</td>
<td style="background-color: inherit;">double</td>
<td style="background-color: inherit;">53</td>
<td style="background-color: inherit;">
Dividends per Share - Ex Date - Daily (Issue)
</td>
</tr>

<tr>
<td style="background-color: inherit;">
divd


<a href="javascript:showMe(&quot;/data-dictionary/comp_na_daily_all/secd/divd/&quot;);"><span class="fa fa-question-circle float-right"></span></a>


</td>
<td style="background-color: inherit;">double</td>
<td style="background-color: inherit;">53</td>
<td style="background-color: inherit;">
Cash Dividends - Daily
</td>
</tr>

<tr>
<td style="background-color: inherit;">
divdpaydate


<a href="javascript:showMe(&quot;/data-dictionary/comp_na_daily_all/secd/divdpaydate/&quot;);"><span class="fa fa-question-circle float-right"></span></a>


</td>
<td style="background-color: inherit;">date</td>
<td style="background-color: inherit;"></td>
<td style="background-color: inherit;">
Cash Dividends - Daily Payment Date
</td>
</tr>

<tr>
<td style="background-color: inherit;">
divdpaydateind


<a href="javascript:showMe(&quot;/data-dictionary/comp_na_daily_all/secd/divdpaydateind/&quot;);"><span class="fa fa-question-circle float-right"></span></a>


</td>
<td style="background-color: inherit;">string</td>
<td style="background-color: inherit;">1</td>
<td style="background-color: inherit;">
Cash Dividends - Daily Payment Date Indicator
</td>
</tr>

<tr>
<td style="background-color: inherit;">
divrc


<a href="javascript:showMe(&quot;/data-dictionary/comp_na_daily_all/secd/divrc/&quot;);"><span class="fa fa-question-circle float-right"></span></a>


</td>
<td style="background-color: inherit;">double</td>
<td style="background-color: inherit;">53</td>
<td style="background-color: inherit;">
None
</td>
</tr>

<tr>
<td style="background-color: inherit;">
divrcpaydate


<a href="javascript:showMe(&quot;/data-dictionary/comp_na_daily_all/secd/divrcpaydate/&quot;);"><span class="fa fa-question-circle float-right"></span></a>


</td>
<td style="background-color: inherit;">date</td>
<td style="background-color: inherit;"></td>
<td style="background-color: inherit;">
None
</td>
</tr>

<tr>
<td style="background-color: inherit;">
divsp


<a href="javascript:showMe(&quot;/data-dictionary/comp_na_daily_all/secd/divsp/&quot;);"><span class="fa fa-question-circle float-right"></span></a>


</td>
<td style="background-color: inherit;">double</td>
<td style="background-color: inherit;">53</td>
<td style="background-color: inherit;">
Special Cash Dividends - Daily
</td>
</tr>

<tr>
<td style="background-color: inherit;">
divsppaydate


<a href="javascript:showMe(&quot;/data-dictionary/comp_na_daily_all/secd/divsppaydate/&quot;);"><span class="fa fa-question-circle float-right"></span></a>


</td>
<td style="background-color: inherit;">date</td>
<td style="background-color: inherit;"></td>
<td style="background-color: inherit;">
Special Cash Dividends - Daily Payment Date
</td>
</tr>

<tr>
<td style="background-color: inherit;">
dvi


<a href="javascript:showMe(&quot;/data-dictionary/comp_na_daily_all/secd/dvi/&quot;);"><span class="fa fa-question-circle float-right"></span></a>


</td>
<td style="background-color: inherit;">double</td>
<td style="background-color: inherit;">53</td>
<td style="background-color: inherit;">
Indicated Annual Dividend - Current
</td>
</tr>

<tr>
<td style="background-color: inherit;">
dvrated


<a href="javascript:showMe(&quot;/data-dictionary/comp_na_daily_all/secd/dvrated/&quot;);"><span class="fa fa-question-circle float-right"></span></a>


</td>
<td style="background-color: inherit;">double</td>
<td style="background-color: inherit;">53</td>
<td style="background-color: inherit;">
Indicated Annual Dividend Rate - Daily
</td>
</tr>

<tr>
<td style="background-color: inherit;">
eps


<a href="javascript:showMe(&quot;/data-dictionary/comp_na_daily_all/secd/eps/&quot;);"><span class="fa fa-question-circle float-right"></span></a>


</td>
<td style="background-color: inherit;">double</td>
<td style="background-color: inherit;">53</td>
<td style="background-color: inherit;">
Current EPS
</td>
</tr>

<tr>
<td style="background-color: inherit;">
epsmo


<a href="javascript:showMe(&quot;/data-dictionary/comp_na_daily_all/secd/epsmo/&quot;);"><span class="fa fa-question-circle float-right"></span></a>


</td>
<td style="background-color: inherit;">double</td>
<td style="background-color: inherit;">53</td>
<td style="background-color: inherit;">
Current EPS Month
</td>
</tr>

<tr>
<td style="background-color: inherit;">
exchg


<a href="javascript:showMe(&quot;/data-dictionary/comp_na_daily_all/secd/exchg/&quot;);"><span class="fa fa-question-circle float-right"></span></a>


</td>
<td style="background-color: inherit;">double</td>
<td style="background-color: inherit;">53</td>
<td style="background-color: inherit;">
Stock Exchange Code
</td>
</tr>

<tr>
<td style="background-color: inherit;">
fic


<a href="javascript:showMe(&quot;/data-dictionary/comp_na_daily_all/secd/fic/&quot;);"><span class="fa fa-question-circle float-right"></span></a>


</td>
<td style="background-color: inherit;">string</td>
<td style="background-color: inherit;">3</td>
<td style="background-color: inherit;">
Current ISO Country Code - Incorporation
</td>
</tr>

<tr>
<td style="background-color: inherit;">
gvkey


<a href="javascript:showMe(&quot;/data-dictionary/comp_na_daily_all/secd/gvkey/&quot;);"><span class="fa fa-question-circle float-right"></span></a>


</td>
<td style="background-color: inherit;">string</td>
<td style="background-color: inherit;">6</td>
<td style="background-color: inherit;">
Global Company Key - Dividends
</td>
</tr>

<tr>
<td style="background-color: inherit;">
iid


<a href="javascript:showMe(&quot;/data-dictionary/comp_na_daily_all/secd/iid/&quot;);"><span class="fa fa-question-circle float-right"></span></a>


</td>
<td style="background-color: inherit;">string</td>
<td style="background-color: inherit;">3</td>
<td style="background-color: inherit;">
Issue ID - Dividends
</td>
</tr>

<tr>
<td style="background-color: inherit;">
paydate


<a href="javascript:showMe(&quot;/data-dictionary/comp_na_daily_all/secd/paydate/&quot;);"><span class="fa fa-question-circle float-right"></span></a>


</td>
<td style="background-color: inherit;">date</td>
<td style="background-color: inherit;"></td>
<td style="background-color: inherit;">
Dividend Payment Date
</td>
</tr>

<tr>
<td style="background-color: inherit;">
paydateind


<a href="javascript:showMe(&quot;/data-dictionary/comp_na_daily_all/secd/paydateind/&quot;);"><span class="fa fa-question-circle float-right"></span></a>


</td>
<td style="background-color: inherit;">string</td>
<td style="background-color: inherit;">1</td>
<td style="background-color: inherit;">
Dividend Payment Date Indicator
</td>
</tr>

<tr>
<td style="background-color: inherit;">
prccd


<a href="javascript:showMe(&quot;/data-dictionary/comp_na_daily_all/secd/prccd/&quot;);"><span class="fa fa-question-circle float-right"></span></a>


</td>
<td style="background-color: inherit;">double</td>
<td style="background-color: inherit;">53</td>
<td style="background-color: inherit;">
Price - Close - Daily
</td>
</tr>

<tr>
<td style="background-color: inherit;">
prchd


<a href="javascript:showMe(&quot;/data-dictionary/comp_na_daily_all/secd/prchd/&quot;);"><span class="fa fa-question-circle float-right"></span></a>


</td>
<td style="background-color: inherit;">double</td>
<td style="background-color: inherit;">53</td>
<td style="background-color: inherit;">
Price - High - Daily
</td>
</tr>

<tr>
<td style="background-color: inherit;">
prcld


<a href="javascript:showMe(&quot;/data-dictionary/comp_na_daily_all/secd/prcld/&quot;);"><span class="fa fa-question-circle float-right"></span></a>


</td>
<td style="background-color: inherit;">double</td>
<td style="background-color: inherit;">53</td>
<td style="background-color: inherit;">
Price - Low - Daily
</td>
</tr>

<tr>
<td style="background-color: inherit;">
prcod


<a href="javascript:showMe(&quot;/data-dictionary/comp_na_daily_all/secd/prcod/&quot;);"><span class="fa fa-question-circle float-right"></span></a>


</td>
<td style="background-color: inherit;">double</td>
<td style="background-color: inherit;">53</td>
<td style="background-color: inherit;">
Price - Open - Daily
</td>
</tr>

<tr>
<td style="background-color: inherit;">
prcstd


<a href="javascript:showMe(&quot;/data-dictionary/comp_na_daily_all/secd/prcstd/&quot;);"><span class="fa fa-question-circle float-right"></span></a>


</td>
<td style="background-color: inherit;">double</td>
<td style="background-color: inherit;">53</td>
<td style="background-color: inherit;">
Price Status Code - Daily
</td>
</tr>

<tr>
<td style="background-color: inherit;">
recorddate


<a href="javascript:showMe(&quot;/data-dictionary/comp_na_daily_all/secd/recorddate/&quot;);"><span class="fa fa-question-circle float-right"></span></a>


</td>
<td style="background-color: inherit;">date</td>
<td style="background-color: inherit;"></td>
<td style="background-color: inherit;">
Dividend Record Date
</td>
</tr>

<tr>
<td style="background-color: inherit;">
secstat


<a href="javascript:showMe(&quot;/data-dictionary/comp_na_daily_all/secd/secstat/&quot;);"><span class="fa fa-question-circle float-right"></span></a>


</td>
<td style="background-color: inherit;">string</td>
<td style="background-color: inherit;">1</td>
<td style="background-color: inherit;">
Security Status Marker
</td>
</tr>

<tr>
<td style="background-color: inherit;">
tic


<a href="javascript:showMe(&quot;/data-dictionary/comp_na_daily_all/secd/tic/&quot;);"><span class="fa fa-question-circle float-right"></span></a>


</td>
<td style="background-color: inherit;">string</td>
<td style="background-color: inherit;">8</td>
<td style="background-color: inherit;">
Ticker Symbol
</td>
</tr>

<tr>
<td style="background-color: inherit;">
tpci


<a href="javascript:showMe(&quot;/data-dictionary/comp_na_daily_all/secd/tpci/&quot;);"><span class="fa fa-question-circle float-right"></span></a>


</td>
<td style="background-color: inherit;">string</td>
<td style="background-color: inherit;">8</td>
<td style="background-color: inherit;">
Issue Type Code
</td>
</tr>

<tr>
<td style="background-color: inherit;">
trfd


<a href="javascript:showMe(&quot;/data-dictionary/comp_na_daily_all/secd/trfd/&quot;);"><span class="fa fa-question-circle float-right"></span></a>


</td>
<td style="background-color: inherit;">double</td>
<td style="background-color: inherit;">53</td>
<td style="background-color: inherit;">
Daily Total Return Factor
</td>
</tr>

</tbody>

</table>

In [29]:
corpus_df = connection.raw_sql(
            '''
                SELECT conm, MAX(anncdate) AS last_divident_announced
                FROM (
                    SELECT *
                    FROM comp.secd
                    WHERE anncdate IS NOT NULL
                    ORDER BY anncdate DESC
                    FETCH FIRST 1000 ROWS ONLY
                ) t
                GROUP BY conm
                ORDER BY last_divident_announced
            '''
        )
corpus_df

,conm,last_divident_announced
0,AMES NATIONAL CORP,2021-11-10
1,BOULDER GROWTH & INC FUND,2021-11-10
2,BOSTON PIZZA ROYALTIES INCM,2021-11-10
3,NACCO INDUSTRIES -CL A,2021-11-10
4,FIRST TRUST MUNI CEF INCM OP,2021-11-10
...,...,...
804,MACKENZIE DE BOND INDEX ETF,2021-11-25
805,MCKNZIE CND SH TR FX INC ETF,2021-11-25
806,MACKENZIE CAN AGRT BND ETF,2021-11-25
807,MACKENZIE GLOBAL FIX INC ETF,2021-11-25


## CRSP Dataset

- [CRSP Daily Stock - Securities (dsf)](https://wrds-www.wharton.upenn.edu/pages/get-data/center-research-security-prices-crsp/annual-update/stock-security-files/daily-stock-file/)
- [CRSP Daily Stock Events - Share Outstanding Information (dse)](https://wrds-www.wharton.upenn.edu/pages/get-data/center-research-security-prices-crsp/annual-update/stock-events/share-outstanding/)
- [CRSP Daily Stock Event -Distribution (dsedist)](https://wrds-www.wharton.upenn.edu/pages/get-data/center-research-security-prices-crsp/annual-update/stock-events/distribution/)

In [8]:
sorted(connection.list_libraries())

['aha',
 'aha_annual_survey_recent',
 'aha_hcris_recent',
 'aha_it_survey_recent',
 'aha_sample',
 'ahasamp',
 'audit',
 'audit_audit_comp',
 'audit_common',
 'audit_corp_legal',
 'auditsmp',
 'auditsmp_all',
 'bank',
 'blab',
 'block',
 'block_all',
 'boardex',
 'boardex_na',
 'boardex_trial',
 'boardsmp',
 'bvd',
 'bvd_ama_large',
 'bvd_ama_medium',
 'bvd_ama_small',
 'bvd_ama_verylarge',
 'bvdsamp',
 'calcbench_trial',
 'calcbnch',
 'cboe',
 'cboe_all',
 'centris',
 'ciq',
 'ciq_capstrct',
 'ciq_common',
 'ciq_keydev',
 'ciq_pplintel',
 'ciqsamp',
 'ciqsamp_common',
 'ciqsamp_transcripts',
 'cisdm',
 'cisdmsmp',
 'comp',
 'comp_bank',
 'comp_bank_daily',
 'comp_execucomp',
 'comp_global',
 'comp_global_daily',
 'comp_na_annual_all',
 'comp_na_daily_all',
 'comp_na_monthly_all',
 'comp_segments_hist',
 'comp_segments_hist_daily',
 'compa',
 'compb',
 'compbd',
 'compdcur',
 'compg',
 'compgd',
 'comph',
 'compm',
 'compmcur',
 'compnad',
 'compsamp',
 'compsamp_snapshot',
 'compseg',

In [10]:
sorted(connection.list_tables('crsp'))

['acti',
 'asia',
 'asib',
 'asic',
 'asio',
 'asix',
 'bmdebt',
 'bmheader',
 'bmpaymts',
 'bmquotes',
 'bmyield',
 'bndprt06',
 'bndprt12',
 'bxcalind',
 'bxdlyind',
 'bxmthind',
 'bxquotes',
 'bxyield',
 'cap',
 'ccm_lookup',
 'ccm_qvards',
 'ccmxpf_linktable',
 'ccmxpf_lnkhist',
 'ccmxpf_lnkrng',
 'ccmxpf_lnkused',
 'comphead',
 'comphist',
 'compmaster',
 'contact_info',
 'crsp_cik_map',
 'crsp_daily_data',
 'crsp_header',
 'crsp_monthly_data',
 'crsp_names',
 'crsp_portno_map',
 'crsp_ziman_daily_index',
 'crsp_ziman_monthly_index',
 'cs20yr',
 'cs5yr',
 'cs90d',
 'cst_hist',
 'daily_nav',
 'daily_nav_ret',
 'daily_returns',
 'dividends',
 'dport1',
 'dport2',
 'dport3',
 'dport4',
 'dport5',
 'dport6',
 'dport7',
 'dport8',
 'dport9',
 'dsbc',
 'dsbo',
 'dse',
 'dse62',
 'dse62delist',
 'dse62dist',
 'dse62exchdates',
 'dse62names',
 'dse62nasdin',
 'dse62shares',
 'dseall',
 'dseall62',
 'dsedelist',
 'dsedist',
 'dseexchdates',
 'dsenames',
 'dsenasdin',
 'dseshares',
 'dsf',


In [21]:
df_stock_splits = connection.raw_sql('''
                    SELECT permno, date, shrout
                    FROM crsp.dse
                    WHERE shrout IS NOT NULL
                    GROUP BY permno, date, shrout
                    FETCH FIRST 1000 ROWS ONLY
                ''',
                date_cols= ['date', 'trtsendt'])
df_stock_splits
# connection.get_table(library="crsp", table= "dse", obs= 100)

,permno,date,shrout
0,10000.0,1986-01-07,3680.0
1,10000.0,1986-01-31,3680.0
2,10000.0,1986-04-30,3793.0
3,10000.0,1986-07-31,3793.0
4,10000.0,1986-10-31,3843.0
...,...,...,...
995,10018.0,1991-04-30,3071.0
996,10018.0,1991-07-31,3071.0
997,10018.0,1991-10-31,3071.0
998,10018.0,1992-01-31,3071.0
